In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("/sps/lsst/users/ebarroso/crow")
from crow.cluster_modules.shear_profile import *
from crow.recipes.murata_binned_spec_z_grid_real import MurataBinnedSpecZRecipeGrid
from crow.recipes.murata_binned_spec_z import MurataBinnedSpecZRecipe
from crow.cluster_modules.mass_proxy import MurataBinned
from crow.cluster_modules.kernel import SpectroscopicRedshift
from crow.cluster_modules.purity import PurityAguena16
from crow.cluster_modules.completeness import CompletenessAguena16
#from firecrown.models.cluster import ClusterProperty
from crow.properties import ClusterProperty
import time
import numpy as np

from scipy.integrate import dblquad, tplquad, simpson

In [3]:
import pyccl as ccl
hmf = ccl.halos.MassFuncTinker08(mass_def="200c")
cosmo = ccl.Cosmology(
    Omega_c=0.2607,      # Cold dark matter density
    Omega_b=0.04897,     # Baryon density
    h=0.6766,            # Hubble parameter
    sigma8=0.8102,       # Matter fluctuation amplitude
    n_s=0.9665,          # Spectral index
)
cl_delta_sigma = ClusterShearProfile(cosmo, hmf, 4.0, True)
#cl_delta_sigma.vectorized= True
pivot_mass, pivot_redshift = 14.625862906, 0.6
comp_dist = CompletenessAguena16()
pur_dist = PurityAguena16()
mass_distribution = MurataBinned(pivot_mass, pivot_redshift)#, pur_dist)
redshift_distribution = SpectroscopicRedshift()



In [4]:
##### Parameters to be used in both recipes #####
mass_points = 30
redshift_points = 10
log_proxy_points = 10
sky_area = 440
mass_interval = (12.5, 15.0)
cluster_theory = cl_delta_sigma
z_bin = (0.2, 0.4)
z_points = np.linspace(z_bin[0], z_bin[1], redshift_points) 
proxy_bin = (1.0, 1.3)
proxy_points = np.linspace(proxy_bin[0], proxy_bin[1], log_proxy_points)
radius_center = 4.0
#################################################

recipe_integral = MurataBinnedSpecZRecipe(
        mass_interval=mass_interval,
        cluster_theory=cluster_theory,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution,
        #completeness=comp_dist,
    )

recipe_grid = MurataBinnedSpecZRecipeGrid(
        mass_interval=mass_interval,
        cluster_theory=cluster_theory,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution,
        #completeness=comp_dist,
    log_proxy_points=log_proxy_points,
    redshift_points=redshift_points,
    log_mass_points=mass_points,
    )

In [5]:
print(recipe_integral.completeness_distribution(np.array([mass_interval[0]]),np.array([z_bin[0]])))

1.0


## Testing mass function

In [6]:
hmf_grid = recipe_grid.get_hmf_grid(z_points, sky_area, (z_points[0], z_points[-1]))
log_mass_grid = recipe_grid.log_mass_grid
def integrand(log_mass_scalar, z_scalar):
    """
    Inputs from dblquad are SCALARS:
    1. log_mass_scalar (y-variable in dblquad)
    2. z_scalar (x-variable in dblquad)
    """
    
    # 1. Convert scalars to single-element arrays to satisfy the methods
    z_array = np.array([z_scalar])
    log_mass_array = np.array([log_mass_scalar])

    # 2. Call the methods with the correct, named arrays
    # comoving_volume(z, sky_area)
    vol_array = recipe_integral.cluster_theory.comoving_volume(z_array, sky_area)
    
    # mass_function(log_mass, z)
    hmf_array = recipe_integral.cluster_theory.mass_function(log_mass_array, z_array)
    
    # 3. Return the result as a scalar float
    return (vol_array * hmf_array)[0]



integral_hmf, error_estimate = dblquad(
    func=integrand, 
    a=z_bin[0],       # lower limit for z (x-axis)
    b=z_bin[1],       # upper limit for z (x-axis)
    gfun=lambda x: mass_interval[0], # lower limit for log_mass (y-axis)
    hfun=lambda x: mass_interval[1]  # upper limit for log_mass (y-axis)
)
integral_over_mass = simpson(
    y=hmf_grid, 
    x=log_mass_grid, 
    axis=1
)

# Step B: Integrate over the redshift dimension (axis=0 of the new array)
# The result is the final scalar integrated number count
simpson_hmf = simpson(
    y=integral_over_mass, 
    x=z_points, 
    axis=0
)

print(f"Simpson Integral {simpson_hmf}, dblquad integral {integral_hmf}")
print(f"Abs error {abs(integral_hmf - simpson_hmf)}, rel error {abs(1.0 - simpson_hmf/integral_hmf)}")



Simpson Integral 87221.31147840535, dblquad integral 87220.6489541402
Abs error 0.6625242651498411, rel error 7.595956612371779e-06


## Testing mass-richness

In [7]:
mass_richness_grid = recipe_grid.get_mass_richness_grid(z_points, proxy_points, None)
###################3
print(f"BECAUSE MASS RICHNESS INTEGRAL HAS PURITY INSIDE, INSTANTIATE A NEW ONE FOR THIS TEST")
mass_distribution_pure = MurataBinned(pivot_mass, pivot_redshift)
recipe_integral_pure = MurataBinnedSpecZRecipe(
        mass_interval=mass_interval,
        cluster_theory=cluster_theory,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution_pure,
        completeness=comp_dist,
    )
################################
def integrand(log_mass_scalar, z_scalar):
    """
    Inputs from dblquad are SCALARS:
    1. log_mass_scalar (y-variable in dblquad)
    2. z_scalar (x-variable in dblquad)
    """
    
    z_array = np.array([z_scalar])
    log_mass_array = np.array([log_mass_scalar])
    return recipe_integral.mass_distribution.distribution(log_mass_array, z_array, proxy_bin)    

z_grid_mesh, log_mass_grid_mesh = np.meshgrid(
            z_points, log_mass_grid, indexing='ij'
        )




integral_mass_richness, error_estimate = dblquad(
    func=integrand, 
    a=z_bin[0],       # lower limit for z (x-axis)
    b=z_bin[1],       # upper limit for z (x-axis)
    gfun=lambda x: mass_interval[0], # lower limit for log_mass (y-axis)
    hfun=lambda x: mass_interval[1]  # upper limit for log_mass (y-axis)
)

integral_over_mass = simpson(
    y=mass_richness_grid, 
    x=log_mass_grid, 
    axis=2
)

integral_over_proxy = simpson(
    y=integral_over_mass, 
    x=proxy_points * np.log(10.0), 
    axis=0
)

simpson_mass_richness = simpson(
    y=integral_over_proxy, 
    x=z_points, 
    axis=0
)

print(f"Simpson Integral {simpson_mass_richness}, dblquad integral {integral_mass_richness}")
print(f"Abs error {abs(integral_mass_richness - simpson_mass_richness)}, rel error {abs(1.0 - simpson_mass_richness/integral_mass_richness)}")



BECAUSE MASS RICHNESS INTEGRAL HAS PURITY INSIDE, INSTANTIATE A NEW ONE FOR THIS TEST
Simpson Integral 0.07492096465752202, dblquad integral 0.0749396525986431
Abs error 1.868794112108718e-05, rel error 0.0002493732019438477


## Testing completeness

In [8]:
comp_grid = recipe_grid.get_completeness_grid(z_points, None)
def integrand(log_mass_scalar, z_scalar):
    z_array = np.array([z_scalar])
    log_mass_array = np.array([log_mass_scalar])
    return recipe_integral.completeness_distribution(log_mass_array, z_array)



integral_hmf, error_estimate = dblquad(
    func=integrand, 
    a=z_bin[0],       # lower limit for z (x-axis)
    b=z_bin[1],       # upper limit for z (x-axis)
    gfun=lambda x: mass_interval[0], # lower limit for log_mass (y-axis)
    hfun=lambda x: mass_interval[1]  # upper limit for log_mass (y-axis)
)
integral_over_mass = simpson(
    y=comp_grid, 
    x=log_mass_grid, 
    axis=1
)

# Step B: Integrate over the redshift dimension (axis=0 of the new array)
# The result is the final scalar integrated number count
simpson_hmf = simpson(
    y=integral_over_mass, 
    x=z_points, 
    axis=0
)

print(f"Simpson Integral {simpson_hmf}, dblquad integral {integral_hmf}")
print(f"Abs error {abs(integral_hmf - simpson_hmf)}, rel error {abs(1.0 - simpson_hmf/integral_hmf)}")



Simpson Integral 0.5, dblquad integral 0.5
Abs error 0.0, rel error 0.0


## Testing Purity

In [9]:
# pur_grid = recipe_grid.compute_purity_grid(z_points, proxy_points)
# def integrand(ln_proxy_scalar, z_scalar):
#     log_proxy_scalar = ln_proxy_scalar / np.log(10.0)
#     z_array = np.array([z_scalar])
#     log_proxy_array = np.array([log_proxy_scalar])
#     return recipe_integral.mass_distribution.purity.distribution(z_array, log_proxy_array)



# integral_hmf, error_estimate = dblquad(
#     func=integrand, 
#     a=z_bin[0],       # lower limit for z (x-axis)
#     b=z_bin[1],       # upper limit for z (x-axis)
#     gfun=lambda x: proxy_bin[0]  * np.log(10.0), # lower limit for log_mass (y-axis)
#     hfun=lambda x: proxy_bin[1] * np.log(10.0), # upper limit for log_mass (y-axis)
# )
# integral_over_proxy = simpson(
#     y=pur_grid, 
#     x=proxy_points, 
#     axis=1
# )

# # Step B: Integrate over the redshift dimension (axis=0 of the new array)
# # The result is the final scalar integrated number count
# simpson_hmf = simpson(
#     y=integral_over_proxy * np.log(10.0), 
#     x=z_points, 
#     axis=0
# )

# print(f"Simpson Integral {simpson_hmf}, dblquad integral {integral_hmf}")
# print(f"Abs error {abs(integral_hmf - simpson_hmf)}, rel error {abs(1.0 - simpson_hmf/integral_hmf)}")



## Testing Shear

In [10]:
shear_grid = recipe_grid.get_shear_grid(z_points, radius_center, None)
log_mass_grid = recipe_grid.log_mass_grid
def integrand(log_mass_scalar, z_scalar):
    """
    Inputs from dblquad are SCALARS:
    1. log_mass_scalar (y-variable in dblquad)
    2. z_scalar (x-variable in dblquad)
    """
    
    # 1. Convert scalars to single-element arrays to satisfy the methods
    z_array = np.array([z_scalar])
    log_mass_array = np.array([log_mass_scalar])
    shear = recipe_integral.cluster_theory.compute_shear_profile(log_mass_array, z_array, radius_center = radius_center)
    
    # 3. Return the result as a scalar float
    return shear[0]



integral_hmf, error_estimate = dblquad(
    func=integrand, 
    a=z_bin[0],       # lower limit for z (x-axis)
    b=z_bin[1],       # upper limit for z (x-axis)
    gfun=lambda x: mass_interval[0], # lower limit for log_mass (y-axis)
    hfun=lambda x: mass_interval[1]  # upper limit for log_mass (y-axis)
)
integral_over_mass = simpson(
    y=shear_grid, 
    x=log_mass_grid, 
    axis=1
)

# Step B: Integrate over the redshift dimension (axis=0 of the new array)
# The result is the final scalar integrated number count
simpson_hmf = simpson(
    y=integral_over_mass, 
    x=z_points, 
    axis=0
)

print(f"Simpson Integral {simpson_hmf}, dblquad integral {integral_hmf}")
print(f"Abs error {abs(integral_hmf - simpson_hmf)}, rel error {abs(1.0 - simpson_hmf/integral_hmf)}")



Simpson Integral 3107547234842.1655, dblquad integral 3107478462523.7495
Abs error 68772318.41601562, rel error 2.2131229305610844e-05


## Testing Counts

In [11]:
recipe_grid.reset_grids_cache()
t1 = time.time()
counts_grid = recipe_grid.evaluate_theory_prediction_base(z_bin, proxy_bin, sky_area)
t2 = time.time()
print(z_bin, proxy_bin)
counts_integral = recipe_integral.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area)
t21 = time.time()
t3 = time.time()
counts_grid_2 = recipe_grid.evaluate_theory_prediction_base(z_bin, proxy_bin, sky_area)
t4 = time.time()
recipe_grid.reset_grids_cache()
t5 = time.time()
counts_grid_3 = recipe_grid.evaluate_theory_prediction_base(z_bin, proxy_bin, sky_area)
t6 = time.time()
print(f"Simpson Integral {counts_grid}, dblquad integral {counts_integral}")
print(f"Abs error {abs(counts_integral - counts_grid)}, rel error {abs(1.0 - counts_grid/counts_integral)}")
print(f"First eval took: {t2-t1}, second eval took: {t4-t3}, integral took: {t21-t2}")
print(f"After reset: {t6-t5}")
print(f" Counts 1, 2 ,3: {counts_grid, counts_grid_2, counts_grid_3}")


(0.2, 0.4) (1.0, 1.3)
Simpson Integral 492.44829165465876, dblquad integral 492.4758087098605
Abs error 0.027517055201712992, rel error 5.587493784475761e-05
First eval took: 0.0013136863708496094, second eval took: 0.0006847381591796875, integral took: 0.04209423065185547
After reset: 0.0011701583862304688
 Counts 1, 2 ,3: (np.float64(492.44829165465876), np.float64(492.44829165465876), np.float64(492.44829165465876))


## Testing DeltaSigma Profile

In [12]:
average_on = ClusterProperty.NONE
average_on |= ClusterProperty.DELTASIGMA
recipe_grid.reset_grids_cache()
t1 = time.time()
counts_grid = recipe_grid.evaluate_theory_prediction_shear_profile(z_edges=z_bin, mass_proxy_edges=proxy_bin, radius_center=radius_center,sky_area=sky_area, average_on=average_on)
t2 = time.time()
counts_integral = recipe_integral.evaluate_theory_prediction_shear_profile(z_edges=z_bin, mass_proxy_edges=proxy_bin, radius_center=radius_center, sky_area=sky_area,average_on=average_on)
t21 = time.time()
t3 = time.time()
counts_grid_2 = recipe_grid.evaluate_theory_prediction_shear_profile(z_edges=z_bin, mass_proxy_edges=proxy_bin, radius_center=radius_center, sky_area=sky_area,average_on=average_on)
t4 = time.time()
recipe_grid.reset_grids_cache()
t5 = time.time()
counts_grid_3 = recipe_grid.evaluate_theory_prediction_shear_profile(z_edges=z_bin, mass_proxy_edges=proxy_bin, radius_center=radius_center, sky_area=sky_area,average_on=average_on)
t6 = time.time()
print(f"Simpson Integral {counts_grid}, dblquad integral {counts_integral}")
print(f"Abs error {abs(counts_integral - counts_grid)}, rel error {abs(1.0 - counts_grid/counts_integral)}")
print(f"First eval took: {t2-t1}, second eval took: {t4-t3}, integral took: {t21-t2}")
print(f"After reset: {t6-t5}")
print(f" Counts 1, 2 ,3: {counts_grid, counts_grid_2, counts_grid_3}")


Simpson Integral 3831544942570779.0, dblquad integral 3831984682268562.0
Abs error 439739697783.0, rel error 0.00011475507713221145
First eval took: 0.12030196189880371, second eval took: 0.0007197856903076172, integral took: 0.12692046165466309
After reset: 0.11747241020202637
 Counts 1, 2 ,3: (np.float64(3831544942570779.0), np.float64(3831544942570779.0), np.float64(3831544942570779.0))


## Testing Reduced Shear Profile

In [13]:
cl_reduced_shear = ClusterShearProfile(cosmo, hmf, 4.0, False, True)
cl_reduced_shear.set_beta_parameters(10)
cl_reduced_shear.set_beta_s_interp(1.1, 1.3)

recipe_integral_reduced = MurataBinnedSpecZRecipe(
        mass_interval=mass_interval,
        cluster_theory=cl_reduced_shear,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution,
        #completeness=comp_dist,
    )

recipe_grid_reduced = MurataBinnedSpecZRecipeGrid(
        mass_interval=mass_interval,
        cluster_theory=cl_reduced_shear,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution,
        #completeness=comp_dist,
    log_proxy_points=log_proxy_points,
    redshift_points=redshift_points,
    log_mass_points=mass_points,
    )

In [14]:
average_on = ClusterProperty.NONE
average_on |= ClusterProperty.DELTASIGMA
recipe_grid.reset_grids_cache()
t1 = time.time()
counts_grid = recipe_grid_reduced.evaluate_theory_prediction_shear_profile(z_edges=z_bin, mass_proxy_edges=proxy_bin, radius_center=radius_center,sky_area=sky_area, average_on=average_on)
t2 = time.time()
counts_integral = recipe_integral_reduced.evaluate_theory_prediction_shear_profile(z_edges=z_bin, mass_proxy_edges=proxy_bin, radius_center=radius_center, sky_area=sky_area,average_on=average_on)
t21 = time.time()
t3 = time.time()
counts_grid_2 = recipe_grid_reduced.evaluate_theory_prediction_shear_profile(z_edges=z_bin, mass_proxy_edges=proxy_bin, radius_center=radius_center, sky_area=sky_area,average_on=average_on)
t4 = time.time()
recipe_grid.reset_grids_cache()
t5 = time.time()
counts_grid_3 = recipe_grid_reduced.evaluate_theory_prediction_shear_profile(z_edges=z_bin, mass_proxy_edges=proxy_bin, radius_center=radius_center, sky_area=sky_area,average_on=average_on)
t6 = time.time()
print(f"Simpson Integral {counts_grid}, dblquad integral {counts_integral}")
print(f"Abs error {abs(counts_integral - counts_grid)}, rel error {abs(1.0 - counts_grid/counts_integral)}")
print(f"First eval took: {t2-t1}, second eval took: {t4-t3}, integral took: {t21-t2}")
print(f"After reset: {t6-t5}")
print(f" Values 1, 2 ,3: {counts_grid, counts_grid_2, counts_grid_3}")


Simpson Integral 1.2280211540730646, dblquad integral 1.360873557658125
Abs error 0.13285240358506045, rel error 0.09762288556306509
First eval took: 0.2346971035003662, second eval took: 0.0007424354553222656, integral took: 0.31749939918518066
After reset: 0.0006551742553710938
 Values 1, 2 ,3: (np.float64(1.2280211540730646), np.float64(1.2280211540730646), np.float64(1.2280211540730646))


## Testing Mean Log Mass Profile

In [15]:
average_on_mass = ClusterProperty.NONE
average_on_mass |= ClusterProperty.MASS

recipe_grid.reset_grids_cache()
t1 = time.time()
counts_grid = recipe_grid.evaluate_theory_prediction_base(z_bin, proxy_bin, sky_area, average_on_mass)
t2 = time.time()
print(z_bin, proxy_bin)
counts_integral = recipe_integral.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area, average_on_mass)
t21 = time.time()
t3 = time.time()
counts_grid_2 = recipe_grid.evaluate_theory_prediction_base(z_bin, proxy_bin, sky_area, average_on_mass)
t4 = time.time()
recipe_grid.reset_grids_cache()
t5 = time.time()
counts_grid_3 = recipe_grid.evaluate_theory_prediction_base(z_bin, proxy_bin, sky_area, average_on_mass)
t6 = time.time()
print(f"Simpson Integral {counts_grid}, dblquad integral {counts_integral}")
print(f"Abs error {abs(counts_integral - counts_grid)}, rel error {abs(1.0 - counts_grid/counts_integral)}")
print(f"First eval took: {t2-t1}, second eval took: {t4-t3}, integral took: {t21-t2}")
print(f"After reset: {t6-t5}")
print(f" Counts 1, 2 ,3: {counts_grid, counts_grid_2, counts_grid_3}")


(0.2, 0.4) (1.0, 1.3)
Simpson Integral 7019.099215264838, dblquad integral 7019.501486555358
Abs error 0.40227129051982047, rel error 5.7307672245765495e-05
First eval took: 0.0013289451599121094, second eval took: 0.0007107257843017578, integral took: 0.0021064281463623047
After reset: 0.0012080669403076172
 Counts 1, 2 ,3: (np.float64(7019.099215264838), np.float64(7019.099215264838), np.float64(7019.099215264838))


## Testing Mean redshift Profile

In [16]:
average_on_mass = ClusterProperty.NONE
average_on_mass |= ClusterProperty.REDSHIFT

recipe_grid.reset_grids_cache()
t1 = time.time()
counts_grid = recipe_grid.evaluate_theory_prediction_base(z_bin, proxy_bin, sky_area, average_on_mass)
t2 = time.time()
print(z_bin, proxy_bin)
counts_integral = recipe_integral.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area, average_on_mass)
t21 = time.time()
t3 = time.time()
counts_grid_2 = recipe_grid.evaluate_theory_prediction_base(z_bin, proxy_bin, sky_area, average_on_mass)
t4 = time.time()
recipe_grid.reset_grids_cache()
t5 = time.time()
counts_grid_3 = recipe_grid.evaluate_theory_prediction_base(z_bin, proxy_bin, sky_area, average_on_mass)
t6 = time.time()
print(f"Simpson Integral {counts_grid}, dblquad integral {counts_integral}")
print(f"Abs error {abs(counts_integral - counts_grid)}, rel error {abs(1.0 - counts_grid/counts_integral)}")
print(f"First eval took: {t2-t1}, second eval took: {t4-t3}, integral took: {t21-t2}")
print(f"After reset: {t6-t5}")
print(f" Counts 1, 2 ,3: {counts_grid, counts_grid_2, counts_grid_3}")


(0.2, 0.4) (1.0, 1.3)
Simpson Integral 153.81441380278108, dblquad integral 153.83237337163465
Abs error 0.017959568853569863, rel error 0.00011674765499580797
First eval took: 0.0012233257293701172, second eval took: 0.0006618499755859375, integral took: 0.0015745162963867188
After reset: 0.0011487007141113281
 Counts 1, 2 ,3: (np.float64(153.81441380278108), np.float64(153.81441380278108), np.float64(153.81441380278108))


## Test speed


In [17]:
mass_distribution_pure = MurataBinned(pivot_mass, pivot_redshift)


##### Parameters to be used in both recipes #####
mass_points = 40
redshift_points = 10
log_proxy_points = 10
sky_area = 440
mass_interval = (12.0, 17.0)
cluster_theory = cl_delta_sigma
z_bin = (0.2, 0.4)
z_points = np.linspace(z_bin[0], z_bin[1], redshift_points) 
proxy_bin = (1.0, 1.3)
proxy_points = np.linspace(proxy_bin[0], proxy_bin[1], log_proxy_points)
radius_center = 4.0
average_on_counts = ClusterProperty.NONE
average_on_shear = ClusterProperty.NONE
average_on_shear |= ClusterProperty.DELTASIGMA
#################################################

recipe_integral_speed = MurataBinnedSpecZRecipe(
        mass_interval=mass_interval,
        cluster_theory=cluster_theory,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution_pure,
        #completeness=comp_dist,
    )

recipe_grid_speed = MurataBinnedSpecZRecipeGrid(
        mass_interval=mass_interval,
        cluster_theory=cluster_theory,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution_pure,
        #completeness=comp_dist,
    log_proxy_points=log_proxy_points,
    redshift_points=redshift_points,
    log_mass_points=mass_points,
    )
recipe_grid_speed.reset_grids_cache()
counts_grid = recipe_grid_speed.evaluate_theory_prediction_base(z_bin, proxy_bin, sky_area,average_on_counts)
counts_integral = recipe_integral_speed.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area, average_on_counts)

counts_grid = recipe_grid_speed.evaluate_theory_prediction_shear_profile(z_bin, proxy_bin, radius_center,sky_area, average_on_shear)
t2 = time.time()
counts_integral = recipe_integral_speed.evaluate_theory_prediction_shear_profile(z_bin, proxy_bin, radius_center, sky_area,average_on_shear)
t21 = time.time()


In [18]:
import time
import numpy as np

# Your parameters
mass_distribution_pure = MurataBinned(pivot_mass, pivot_redshift)

mass_points = 40
redshift_points = 12
log_proxy_points = 12
sky_area = 440
mass_interval = (12.0, 17.0)
z_bin = (0.2, 0.4)
proxy_bin = (1.0, 1.3)
radius_center = 4.0

average_on_counts = ClusterProperty.NONE
average_on_shear  = ClusterProperty.DELTASIGMA

# Recipes
recipe_integral_speed = MurataBinnedSpecZRecipe(
    mass_interval=mass_interval,
    cluster_theory=cl_delta_sigma,
    redshift_distribution=redshift_distribution,
    mass_distribution=mass_distribution_pure,
    #completeness=comp_dist,
)

recipe_grid_speed = MurataBinnedSpecZRecipeGrid(
    mass_interval=mass_interval,
    cluster_theory=cl_delta_sigma,
    redshift_distribution=redshift_distribution,
    mass_distribution=mass_distribution_pure,
    #completeness=comp_dist,
    log_proxy_points=log_proxy_points,
    redshift_points=redshift_points,
    log_mass_points=mass_points,
)

# ---- One-shot timing: counts ----
recipe_grid_speed.reset_grids_cache()

t0 = time.time()
counts_grid = recipe_grid_speed.evaluate_theory_prediction_base(
    z_bin, proxy_bin, sky_area, average_on_counts
)
t1 = time.time()

counts_integral = recipe_integral_speed.evaluate_theory_prediction_counts(
    z_bin, proxy_bin, sky_area, average_on_counts
)
t2 = time.time()

print("\n--- One-shot counts ---")
print("Grid time:    %.4f s" % (t1 - t0))
print("Integral time: %.4f s" % (t2 - t1))


# ---- One-shot timing: shear ----
t0 = time.time()
shear_grid = recipe_grid_speed.evaluate_theory_prediction_shear_profile(
    z_bin, proxy_bin, radius_center, sky_area, average_on_shear
)
t1 = time.time()

shear_integral = recipe_integral_speed.evaluate_theory_prediction_shear_profile(
    z_bin, proxy_bin, radius_center, sky_area, average_on_shear
)
t2 = time.time()

print("\n--- One-shot shear ---")
print("Grid time:    %.4f s" % (t1 - t0))
print("Integral time: %.4f s" % (t2 - t1))



--- One-shot counts ---
Grid time:    0.0508 s
Integral time: 0.0022 s

--- One-shot shear ---
Grid time:    0.1892 s
Integral time: 0.4594 s


In [19]:
richness_bins = [(0.3,0.4), (0.4,0.6), (0.6,0.8), (0.8,1.0)]   # example
proxy_bins    = [(1.0,1.1), (1.1,1.2), (1.2,1.3), (1.3,1.4), (1.4,1.5)]
radii         = np.linspace(1.0, 10.0, 4)

# ----------------------------------------------
# Total GRID computation time
# ----------------------------------------------
recipe_grid_speed.reset_grids_cache()
counts_grid_arr = []
shear_grid_arr = []
tg0 = time.time()
for rbin in richness_bins:
    for pbin in proxy_bins:
        for R in radii:
            counts_grid_arr.append(recipe_grid_speed.evaluate_theory_prediction_base(rbin, pbin, sky_area, average_on_counts))
            shear_grid_arr.append(recipe_grid_speed.evaluate_theory_prediction_shear_profile(rbin, pbin, R, sky_area, average_on_shear))
tg1 = time.time()
print("\nTOTAL GRID TIME = %.2f s" % (tg1 - tg0))


# ----------------------------------------------
# Total INTEGRAL computation time
# ----------------------------------------------
counts_int_arr = []
shear_int_arr = []
ti0 = time.time()
for rbin in richness_bins:
    for pbin in proxy_bins:
        for R in radii:
            counts_int_arr.append(recipe_integral_speed.evaluate_theory_prediction_counts(rbin, pbin, sky_area, average_on_counts))
            shear_int_arr.append(recipe_integral_speed.evaluate_theory_prediction_shear_profile(rbin, pbin, R, sky_area, average_on_shear))
ti1 = time.time()
print("TOTAL INTEGRAL TIME = %.2f s" % (ti1 - ti0))

# ----------------------------------------------
# Comparison of Results
# ----------------------------------------------

# Convert lists to NumPy arrays for easy calculation
counts_grid = np.array(counts_grid_arr)
counts_int = np.array(counts_int_arr)
shear_grid = np.array(shear_grid_arr)
shear_int = np.array(shear_int_arr)

counts_diff = np.where(
    counts_int != 0.0, 
    np.abs(counts_grid - counts_int) / np.abs(counts_int), 
    0.0
)
rms_counts_diff = np.sqrt(np.mean(counts_diff**2))

shear_diff = np.where(
    shear_int != 0.0, 
    np.abs(shear_grid - shear_int) / np.abs(shear_int), 
    0.0
)
rms_shear_diff = np.sqrt(np.mean(shear_diff**2))

# Final Comparison Printout
print("\n--- RESULTS COMPARISON ---")
print("Total number of bins computed: %d" % len(counts_grid))
print("RMS Relative Diff (Counts): %.4e" % rms_counts_diff)
print("RMS Relative Diff (Shear):  %.4e" % rms_shear_diff)
print("--------------------------")



TOTAL GRID TIME = 3.34 s
TOTAL INTEGRAL TIME = 26.71 s

--- RESULTS COMPARISON ---
Total number of bins computed: 80
RMS Relative Diff (Counts): 3.0806e-04
RMS Relative Diff (Shear):  3.7307e-04
--------------------------


In [20]:
print(average_on_shear)

ClusterProperty.DELTASIGMA


In [21]:
def frac_diff(a, b):
    return np.abs(a - b) / (np.abs(b) + 1e-12)


print("\n===== BEGIN ACCURACY SWEEP WITH COUNTS =====\n")

# --- ranges to test ---
proxy_sweep    = [2, 5, 10, 20, 30, 40, 60, 80, 120, 160]
z_sweep        = [2, 5, 10, 20, 30, 40]
mass_sweep     = [5, 10, 20, 40, 60, 80, 120]

# --- fixed test bin ---
z_lo, z_hi = z_bin
proxy_lo, proxy_hi = proxy_bin
R = radius_center

print("Test bin:")
print("  z = (%.3f, %.3f)" % z_bin)
print("  proxy = (%.3f, %.3f)" % proxy_bin)
print("  R = %.3f" % R)


# --------------------------------------------------------------
# 1) PROXY SWEEP
# --------------------------------------------------------------
print("\n---- Sweep log_proxy_points ----")

for pts in proxy_sweep:
    print(f"\nTrying log_proxy_points = {pts}")

    recipe_grid_speed_pts = MurataBinnedSpecZRecipeGrid(
        mass_interval=mass_interval,
        cluster_theory=cl_delta_sigma,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution_pure,
        #completeness=comp_dist,
        log_proxy_points=pts,
        redshift_points=redshift_points,
        log_mass_points=mass_points,
    )
    recipe_grid_speed_pts.reset_grids_cache()
    
    # Shear evaluation
    val_shear = recipe_grid_speed_pts.evaluate_theory_prediction_shear_profile(
        z_bin, proxy_bin, R, sky_area, average_on_shear
    )
    
    # Counts evaluation
    counts_val = recipe_grid_speed_pts.evaluate_theory_prediction_base(
        z_bin, proxy_bin, sky_area
    )
    
    recipe_grid_speed_pts.reset_grids_cache()
    
    # Reference shear
    ref_shear = recipe_integral_speed.evaluate_theory_prediction_shear_profile(
        z_bin, proxy_bin, R, sky_area, average_on_shear
    )
    
    # Reference counts
    ref_counts = recipe_integral_speed.evaluate_theory_prediction_counts(
        z_bin, proxy_bin, sky_area
    )
    
    diff_shear  = float(frac_diff(val_shear, ref_shear))
    diff_counts = float(frac_diff(counts_val, ref_counts))

    print("  shear diff  = %.6f" % diff_shear)
    print("  counts diff = %.6f" % diff_counts)

    if diff_shear < 0.01 and diff_counts < 0.01:
        print("  → Achieved ≤1% accuracy at log_proxy_points =", pts)
        break


# --------------------------------------------------------------
# 2) REDSHIFT SWEEP
# --------------------------------------------------------------
print("\n---- Sweep redshift_points ----")

for pts in z_sweep:
    print(f"\nTrying redshift_points = {pts}")

    recipe_grid_speed_pts = MurataBinnedSpecZRecipeGrid(
        mass_interval=mass_interval,
        cluster_theory=cl_delta_sigma,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution_pure,
        #completeness=comp_dist,
        log_proxy_points=log_proxy_points,
        redshift_points=pts,
        log_mass_points=mass_points,
    )
    recipe_grid_speed_pts.reset_grids_cache()
    
    val_shear = recipe_grid_speed_pts.evaluate_theory_prediction_shear_profile(
        z_bin, proxy_bin, R, sky_area, average_on_shear
    )
    counts_val = recipe_grid_speed_pts.evaluate_theory_prediction_base(
        z_bin, proxy_bin, sky_area
    )

    ref_shear = recipe_integral_speed.evaluate_theory_prediction_shear_profile(
        z_bin, proxy_bin, R, sky_area, average_on_shear
    )
    ref_counts = recipe_integral_speed.evaluate_theory_prediction_counts(
        z_bin, proxy_bin, sky_area
    )
    
    diff_shear  = float(frac_diff(val_shear, ref_shear))
    diff_counts = float(frac_diff(counts_val, ref_counts))
    
    print("  shear diff  = %.6f" % diff_shear)
    print("  counts diff = %.6f" % diff_counts)

    if diff_shear < 0.01 and diff_counts < 0.01:
        print("  → Achieved ≤1% accuracy at redshift_points =", pts)
        break


# --------------------------------------------------------------
# 3) MASS SWEEP
# --------------------------------------------------------------
print("\n---- Sweep log_mass_points ----")

for pts in mass_sweep:
    print(f"\nTrying log_mass_points = {pts}")

    recipe_grid_speed_pts = MurataBinnedSpecZRecipeGrid(
        mass_interval=mass_interval,
        cluster_theory=cl_delta_sigma,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution_pure,
        #completeness=comp_dist,
        log_proxy_points=log_proxy_points,
        redshift_points=redshift_points,
        log_mass_points=pts,
    )
    recipe_grid_speed_pts.reset_grids_cache()
    
    val_shear = recipe_grid_speed_pts.evaluate_theory_prediction_shear_profile(
        z_bin, proxy_bin, R, sky_area, average_on_shear
    )
    counts_val = recipe_grid_speed_pts.evaluate_theory_prediction_base(
        z_bin, proxy_bin,  sky_area
    )

    ref_shear = recipe_integral_speed.evaluate_theory_prediction_shear_profile(
        z_bin, proxy_bin, R, sky_area, average_on_shear
    )
    ref_counts = recipe_integral_speed.evaluate_theory_prediction_counts(
        z_bin, proxy_bin, sky_area
    )
    
    diff_shear  = float(frac_diff(val_shear, ref_shear))
    diff_counts = float(frac_diff(counts_val, ref_counts))
    
    print("  shear diff  = %.6f" % diff_shear)
    print("  counts diff = %.6f" % diff_counts)

    # Optional early break if both ≤1%
    # if diff_shear < 0.01 and diff_counts < 0.01:
    #     print("  → Achieved ≤1% accuracy at log_mass_points =", pts)
    #     break


print("\n===== END ACCURACY SWEEP WITH COUNTS =====\n")



===== BEGIN ACCURACY SWEEP WITH COUNTS =====

Test bin:
  z = (0.200, 0.400)
  proxy = (1.000, 1.300)
  R = 4.000

---- Sweep log_proxy_points ----

Trying log_proxy_points = 2
  shear diff  = 0.024303
  counts diff = 0.148385

Trying log_proxy_points = 5
  shear diff  = 0.000034
  counts diff = 0.000131
  → Achieved ≤1% accuracy at log_proxy_points = 5

---- Sweep redshift_points ----

Trying redshift_points = 2
  shear diff  = 0.030061
  counts diff = 0.036235

Trying redshift_points = 5
  shear diff  = 0.000044
  counts diff = 0.000093
  → Achieved ≤1% accuracy at redshift_points = 5

---- Sweep log_mass_points ----

Trying log_mass_points = 5
  shear diff  = 0.109240
  counts diff = 0.261848

Trying log_mass_points = 10
  shear diff  = 0.225185
  counts diff = 0.173935

Trying log_mass_points = 20
  shear diff  = 0.041281
  counts diff = 0.003686

Trying log_mass_points = 40
  shear diff  = 0.000037
  counts diff = 0.000082

Trying log_mass_points = 60
  shear diff  = 0.000007
  c

## Compare vectorization times

In [22]:
recipe_grid = MurataBinnedSpecZRecipeGrid(
    mass_interval=mass_interval,
    cluster_theory=cl_delta_sigma,
    redshift_distribution=redshift_distribution,
    mass_distribution=mass_distribution,
    completeness=comp_dist,
    log_proxy_points=10,
    redshift_points=11,
    log_mass_points=12,
)

In [23]:
proxy_bins = np.linspace(0.5, 1.5, 5)
z_bins = np.linspace(0.2, 1.2, 11)
r_vals = np.logspace(-1, 1,  10)

average_on_counts = ClusterProperty.NONE
average_on_shear = ClusterProperty.NONE
average_on_shear |= ClusterProperty.DELTASIGMA

### One halo term

In [24]:
# Old computation
recipe_grid.cluster_theory.vectorized = False
t0 = time.time()
recipe_grid.reset_grids_cache()
val_shear_1h = np.zeros(
    (
        proxy_bins.size-1,
        z_bins.size-1,
        r_vals.size,            
    )
)
for i in range(proxy_bins.size-1):
    for j in range(z_bins.size-1):
        for k, r in enumerate(r_vals):
            val_shear_1h[i, j, k] = recipe_grid.evaluate_theory_prediction_shear_profile(
                z_bins[j:j+2], proxy_bins[i:i+2], r, 440, average_on_shear
            )
time_old = time.time()-t0
print(f"old computation: {time_old:.3f} seconds")

# Vectorized computation

t0 = time.time()
recipe_grid.reset_grids_cache()
val_shear_1h_vec = np.zeros(
    (
        proxy_bins.size-1,
        z_bins.size-1,
        r_vals.size,            
    )
)
for i in range(proxy_bins.size-1):
    for j in range(z_bins.size-1):
        val_shear_1h_vec[i, j, :] = recipe_grid.evaluate_theory_prediction_shear_profile_vectorized(
                z_bins[j:j+2], proxy_bins[i:i+2], r_vals, 440, average_on_shear
            )
time_vec = time.time()-t0
print(f"vec computation: {time_vec:.3f} seconds")


print()
print(f"speed up factor   : {time_old/time_vec:.1f}")
print(f"max relative diff : {(val_shear_1h_vec/val_shear_1h-1).max():.2g}")

old computation: 5.523 seconds
vec computation: 0.058 seconds

speed up factor   : 94.6
max relative diff : 2.2e-16


### Two halo term

The integration routine was changed, so first we make sure they are consistent

In [25]:
import clmm

In [26]:
from crow.cluster_modules._clmm_patches import (
    _eval_2halo_term_generic_new,
    _eval_2halo_term_generic_orig,
    _eval_2halo_term_generic_vec,
)

In [27]:
# Original computation
clmm.Modeling._eval_2halo_term_generic = _eval_2halo_term_generic_orig
t0 = time.time()
recipe_grid.cluster_theory.vectorized = False
recipe_grid.cluster_theory.two_halo_term = True
recipe_grid.cluster_theory.boost_factor = False
recipe_grid.reset_grids_cache()
val_shear_2h_orig = np.zeros(
    (
        proxy_bins.size-1,
        z_bins.size-1,
        r_vals.size,            
    )
)
for i in range(proxy_bins.size-1):
    for j in range(z_bins.size-1):
        for k, r in enumerate(r_vals):
            val_shear_2h_orig[i, j, k] = recipe_grid.evaluate_theory_prediction_shear_profile(
                z_bins[j:j+2], proxy_bins[i:i+2], r, 440, average_on_shear
            )
time_old = time.time()-t0
print(f"old computation: {time_old:.3f} seconds")


excess_val_shear_2h_orig = val_shear_2h_orig-val_shear_1h

old computation: 21.758 seconds


In [28]:
clmm.Modeling._eval_2halo_term_generic = _eval_2halo_term_generic_new
t0 = time.time()
recipe_grid.cluster_theory.two_halo_term = True
recipe_grid.cluster_theory.vectorized = False
recipe_grid.reset_grids_cache()
val_shear_2h_new = np.zeros(
    (
        proxy_bins.size-1,
        z_bins.size-1,
        r_vals.size,            
    )
)
for i in range(proxy_bins.size-1):
    for j in range(z_bins.size-1):
        for k, r in enumerate(r_vals):
            val_shear_2h_new[i, j, k] = recipe_grid.evaluate_theory_prediction_shear_profile(
                z_bins[j:j+2], proxy_bins[i:i+2], r, 440, average_on_shear
            )
time_old = time.time()-t0
print(f"old computation: {time_old:.3f} seconds")

excess_val_shear_2h_new = val_shear_2h_new-val_shear_1h
print(f"max relative diff : {(excess_val_shear_2h_new/excess_val_shear_2h_orig-1).max():.2g}")

old computation: 21.595 seconds
max relative diff : 0


In [29]:
clmm.Modeling._eval_2halo_term_generic = _eval_2halo_term_generic_vec
t0 = time.time()
recipe_grid.cluster_theory.two_halo_term = True
recipe_grid.cluster_theory.vectorized = False
recipe_grid.reset_grids_cache()
val_shear_2h_new2 = np.zeros(
    (
        proxy_bins.size-1,
        z_bins.size-1,
        r_vals.size,            
    )
)
for i in range(proxy_bins.size-1):
    for j in range(z_bins.size-1):
        for k, r in enumerate(r_vals):
            val_shear_2h_new2[i, j, k] = recipe_grid.evaluate_theory_prediction_shear_profile(
                z_bins[j:j+2], proxy_bins[i:i+2], r, 440, average_on_shear
            )
time_old = time.time()-t0
print(f"old computation: {time_old:.3f} seconds")

excess_val_shear_2h_new2 = val_shear_2h_new2-val_shear_1h
print(f"max relative diff : {(excess_val_shear_2h_new2/excess_val_shear_2h_orig-1).max():.2g}")

old computation: 15.965 seconds
max relative diff : 4.1e-07


In [33]:
clmm.Modeling._eval_2halo_term_generic = _eval_2halo_term_generic_vec
t0 = time.time()
recipe_grid.cluster_theory.two_halo_term = True
recipe_grid.cluster_theory.boost_factor = False
recipe_grid.cluster_theory.vectorized = False
recipe_grid.reset_grids_cache()
val_shear_2h_vec = np.zeros(
    (
        proxy_bins.size-1,
        z_bins.size-1,
        r_vals.size,            
    )
)
for i in range(proxy_bins.size-1):
    for j in range(z_bins.size-1):
        val_shear_2h_vec[i, j, :] = recipe_grid.evaluate_theory_prediction_shear_profile_vectorized(
            z_bins[j:j+2], proxy_bins[i:i+2], r_vals, 440, average_on_shear
            )
time_vec = time.time()-t0
print(f"vec computation: {time_vec:.3f} seconds")

excess_val_shear_2h_vec = val_shear_2h_vec-val_shear_1h_vec
print(f"max relative diff (new ) : {(excess_val_shear_2h_vec/excess_val_shear_2h_new-1).max():.2g}")
print(f"max relative diff (new2) : {(excess_val_shear_2h_vec/excess_val_shear_2h_new2-1).max():.2g}")
print(f"max relative diff (new2 vs orig) : {(excess_val_shear_2h_vec/excess_val_shear_2h_orig-1).max():.2g}")

vec computation: 0.339 seconds
max relative diff (new ) : 4.1e-07
max relative diff (new2) : 4e-13
max relative diff (new2 vs orig) : 4.1e-07


### Boost factor

In [31]:
t0 = time.time()
recipe_grid.cluster_theory.two_halo_term = True
recipe_grid.cluster_theory.boost_factor = True
recipe_grid.cluster_theory.vectorized = False
recipe_grid.reset_grids_cache()
val_shear_2h_boost = np.zeros(
    (
        proxy_bins.size-1,
        z_bins.size-1,
        r_vals.size,            
    )
)
for i in range(proxy_bins.size-1):
    for j in range(z_bins.size-1):
        for k, r in enumerate(r_vals):
            val_shear_2h_boost[i, j, k] = recipe_grid.evaluate_theory_prediction_shear_profile(
                z_bins[j:j+2], proxy_bins[i:i+2], r, 440, average_on_shear
            )
time_old = time.time()-t0
print(f"old computation: {time_old:.3f} seconds")

t0 = time.time()
recipe_grid.reset_grids_cache()
val_shear_2h_boost_vec = np.zeros(
    (
        proxy_bins.size-1,
        z_bins.size-1,
        r_vals.size,            
    )
)
for i in range(proxy_bins.size-1):
    for j in range(z_bins.size-1):
        val_shear_2h_boost_vec[i, j, :] = recipe_grid.evaluate_theory_prediction_shear_profile_vectorized(
            z_bins[j:j+2], proxy_bins[i:i+2], r_vals, 440, average_on_shear
            )
time_vec = time.time()-t0
print(f"vec computation: {time_vec:.3f} seconds")

print()
print(f"speed up factor   : {time_old/time_vec:.1f}")
print(f"max relative diff : {(val_shear_2h_boost_vec/val_shear_2h_boost-1).max():.2g}")

old computation: 16.120 seconds
vec computation: 0.337 seconds

speed up factor   : 47.8
max relative diff : 4.4e-16


In [32]:
t0 = time.time()
recipe_grid.cluster_theory.two_halo_term = False
recipe_grid.cluster_theory.boost_factor = True
recipe_grid.cluster_theory.vectorized = False
recipe_grid.reset_grids_cache()
val_shear_2h_boost = np.zeros(
    (
        proxy_bins.size-1,
        z_bins.size-1,
        r_vals.size,            
    )
)
for i in range(proxy_bins.size-1):
    for j in range(z_bins.size-1):
        for k, r in enumerate(r_vals):
            val_shear_2h_boost[i, j, k] = recipe_grid.evaluate_theory_prediction_shear_profile(
                z_bins[j:j+2], proxy_bins[i:i+2], r, 440, average_on_shear
            )
time_old = time.time()-t0
print(f"old computation: {time_old:.3f} seconds")

t0 = time.time()
recipe_grid.reset_grids_cache()
val_shear_2h_boost_vec = np.zeros(
    (
        proxy_bins.size-1,
        z_bins.size-1,
        r_vals.size,            
    )
)
for i in range(proxy_bins.size-1):
    for j in range(z_bins.size-1):
        val_shear_2h_boost_vec[i, j, :] = recipe_grid.evaluate_theory_prediction_shear_profile_vectorized(
            z_bins[j:j+2], proxy_bins[i:i+2], r_vals, 440, average_on_shear
            )
time_vec = time.time()-t0
print(f"vec computation: {time_vec:.3f} seconds")

print()
print(f"speed up factor   : {time_old/time_vec:.1f}")
print(f"max relative diff : {(val_shear_2h_boost_vec/val_shear_2h_boost-1).max():.2g}")

old computation: 5.500 seconds
vec computation: 0.058 seconds

speed up factor   : 94.2
max relative diff : 2.2e-16
